## Downloading FLamby datasets
See tutorial Flamby integration in Fed-BioMed


### Install dependencies

In [17]:
! pip install wget nibabel # lecture files format .nii

In [1]:
pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import cv2

In [20]:
import numpy as np
import pandas as pd
import os
from PIL import Image 
from torch.utils.data import Dataset 
from torchvision import datasets, transforms, models
import torchvision.transforms as transforms # A module from the torchvision library that provides common image transformations, such as resizing, cropping, and normalization.
from torch.utils.data import random_split # A function from PyTorch that allows for randomly splitting a dataset into training and validation subsets.

import torch.nn as nn # A module in PyTorch that provides classes for defining and building neural networks.
from torchvision import utils # A module from torchvision that contains utility functions for working with images, such as saving and visualizing them.
from torch.utils.data import Dataset

import torch.optim as optim
#import splitfolders
from torchsummary import summary
#import torch.nn.functional as F
import pathlib

import itertools # This import statement imports the itertools module, which provides functions for efficient looping and combining of iterables. It can be used for tasks such as generating combinations or permutations of elements.
from tqdm.notebook import trange, tqdm # These functions allow for the creation of progress bars to track the progress of loops or tasks.
from torch import optim
import warnings
warnings.filterwarnings('ignore')


In [21]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


## Loads data

### Loads the model 

In [6]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /user/ebirgy/home/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-1-19 Python-3.10.13 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 3906MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [9]:
model.eval()

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

## Basic example: Fed-IXI

The first example will use the model, optimizer and loss function provided by FLamby for the IXI dataset.

The instructions for using FLamby are:

    define a TorchTrainingPlan
    in the training_data function, instantiate a FlambyDataset
    make sure to include the necessary dependencies in the init_dependencies function

